# [Can You Make Something out of Nothing?](https://thefiddler.substack.com/p/can-you-make-something-out-of-nothing)
## July 28 2023

# Problem

_From Jeremy Dixon comes a tale of two functions:_

_Consider f(n) = 2n+1 and g(n) = 4n. It’s possible to produce different whole numbers by applying combinations of f and g to 0. For example, f(0) = 1, f(f(0)) = 3, g(f(0)) = 4, f(f(f(0))) = 7, f(g(f(0))) = 9, f(f(f(f(0)))) = 15, and g(g(f(0))) = 16._

_How many whole numbers between 1 and 1,024 (including 1 and 1,024) can you produce by applying some combination of f’s and g’s to the number 0?_

# Solution

Let's first examine some of the convolutions of $f$ and $g$.
 - $(f * g)(n) = 8n + 1$
 - $(g * f)(n) = 8n + 4$
 - $(f * f)(n) = 4n + 3$
 - $(g * g)(n) = 16n$

Interestingly, none of the convolutions are capable of generating the same number regardless of what $n$ is given as input. If we can prove that this is the case with all possible convolutions of $f$ and $g$ then we won't have to worry about different convolutions generating the same number and we can simplify the problem to 'how many convolutions can we generate that stay within our range $[1,1024]$, which is much easier. Let's see if we can prove it. 

Let $ \psi (x) = (a_n * a_{n-1} * ... * a_i * ... * a_1)(x) $ and $ \phi (x) = (b_m * b_{m-1} * ... * b_i * ... * b_1)(x) $ where $a_i$ and $b_i$ are either $f$ or $g$. Assume $\psi(0) = \phi(0)$.

If $\psi(0) = \phi(0)$, then $\psi$ and $\phi$ must both either have $f$ or $g$ for their final function since $f$ is only capable of generating odd numbers and $g$ is only capable of generating even numbers. Let's call this final function $a_n$. In other words $a_n = b_m$.

Since $f$ and $g$ are both bijective functions, we know $a_n^{-1}(\psi(0)) = a_n^{-1}(\phi(0))$.

But we can use the same logic as before to deduce that $a_n^{-1}(\psi(0)) $ and $ a_n^{-1}(\phi(0))$ must have the same last function. In this way, we can unravel all of $\psi$ and $\phi$ to show that they must be the same convolution if they yield the same value.

So each unique convolution generates a different number.  

With that in hand, all we need to do is determine how many different convolutions of $f$ and $g$ remain within our range of 1024.  Answering this is made much easier by noting that $f$ and $g$ are both monotonically increasing functions and that $f$ approxiately doubles $n$ while $g$ quadruples $n$. 

Interestingly, this is very similar a classic algorithms problem - the [Climbing Stairs](https://www.geeksforgeeks.org/count-ways-reach-nth-stair/) problem. In this problem you have to count the number of unque ways to get the the nth step by moving up by either 1 or 2 steps at a time. In this case, moving up 1 step is the same as doubling our input.  Similarily, moving up 2 steps would quadrupling. We can perform as most $\log_2 (1024) + 1 = 11$ doublings if we start at $f(0)=1$. More succinctly:

 - $f$ represents 1 step, 
 - $g$ represents 2 steps, and 
 - we want to count the number of different ways to get up to $\log_2 (1024) + 1 = 11$ steps. 
 
We can use a dynamic programming approch to this calculation by noting that you can get to the $i^{th}$ step 2 ways: from $i-1$ step or the $i-2$ step. This gives us the recurrence relation:

$ N(i) = N(i-1) + N(i-2)$

$ N(0) = 1, N(1) = 1 $

The code below calculates this number and the verifies the solution by actually generating every possible convolution. We get an answer of *144*.



In [8]:
import math

def recur(n):
    if n == 0:
        return 1
    if n == 1:
        return 1

    return recur(n-1) + recur(n-2)

print(recur(math.log2(1024)+1))

144


In [9]:
# Use this code to verify the answer
def f1(n:int):
  return 2*n + 1

def f2(n:int):
  return 4*n

def range_check(n):
  return n>=1 and n<=1024

unexplored = [0]
explored = set()
while len(unexplored) > 0:
  n = unexplored.pop()
  explored.add(n)
  x = f1(n)
  y = f2(n)
  if x not in explored and range_check(x):
    unexplored.append(x)
  if y not in explored and range_check(y):
    unexplored.append(y)
explored.discard(0)

print(len(explored))

144


# Extra Credit

_Now consider the functions g(n) = 4n and h(n) = 1−2n. How many integers between -1,024 and 1,024 (including -1,024 and 1,024) can you produce by applying some combination of g’s and h’s to the number 0?_

# Solution

For the extra credit, we replace $f$ with $h(n)=1-2n$. Since $h$ is still only cabable of generating odd numbers (including negatives), we should be able to apply the same logic as above in order to conclude that every unique value generated from convolutions of $f$ and $h$ can only come from that particular convolution. 

Note that, like $f$, $h$ also approximately doubles its input. Unlike $f$, $h$ causes its input to oscillate between positive and negative values.  In order to properly tranlate this situation into n-steps problem, I'll note the symmetry with the original version.  In both cases, $f$ and $h$ approximately double the input while $g$ quadruples it and we're still concerned with keeping it within $\pm 1024$.  Now however, there is are two ways of advancing: in the negative direction and the positive direction.  We can capture this by adjusting our base case to reflect that there are now 2 ways to get to $\pm 1$: 
 - $h(0)=1$ and 
 - $h(h(0))=-1$

So if we adjust our base case to $N(1) = 2$, we should get our answer. This yields *233* which, after checking against the brute force approach, appears to be the correct answer!

In [11]:
def recur(n):
    if n == 0:
        return 1
    if n == 1:
        return 2

    return recur(n-1) + recur(n-2)

print(recur(math.log2(1024)+1))

233


In [10]:
# Use this code to verify the answer
def f1(n:int):
  return 1-2*n

def f2(n:int):
  return 4*n

def range_check(n):
  return n>=-1024 and n<=1024

unexplored = [0]
explored = set()
while len(unexplored) > 0:
  n = unexplored.pop()
  explored.add(n)
  x = f1(n)
  y = f2(n)
  if x not in explored and range_check(x):
    unexplored.append(x)
  if y not in explored and range_check(y):
    unexplored.append(y)
explored.discard(0)

print(len(explored))

233
